In [1]:
cd projects/pytorch/

[Errno 2] No such file or directory: 'projects/pytorch/'
/home/candy/mlos/projects/pytorch


In [ ]:

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Load data
df = pd.read_csv("heart.csv")

# Preprocess data
x=features = df.drop("target", axis=1)
y=labels = df["target"]

# Normalize numerical data
scaler = StandardScaler()

In [3]:

numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
features[numerical_cols] = scaler.fit_transform(features[numerical_cols])

In [4]:

#encode categorical data 
encoder = OneHotEncoder()
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

In [6]:
# Fit and transform the categorical data
categorical_data_transformed = encoder.fit_transform(features[categorical_cols])
# Convert the sparse matrix to a dense array
categorical_data_transformed_array = categorical_data_transformed.toarray()
# Get the names of the new columns generated by the OneHotEncoder
new_cols = encoder.get_feature_names_out(categorical_cols)
# Create a DataFrame from the dense array
categorical_df = pd.DataFrame(categorical_data_transformed_array, columns=new_cols)
# Drop the original categorical columns from 'features'
features = features.drop(columns=categorical_cols)
# Concatenate the original DataFrame 'features' with the new DataFrame 'categorical_df'
features = pd.concat([features, categorical_df], axis=1)


In [8]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [11]:

# Convert data to tensors
features_train = torch.tensor(features_train.values, dtype=torch.float)
labels_train = torch.tensor(labels_train.values, dtype=torch.long)
features_test = torch.tensor(features_test.values, dtype=torch.float)
labels_test = torch.tensor(labels_test.values, dtype=torch.long)

In [12]:

# Create dataloaders
train_data = TensorDataset(features_train, labels_train)
test_data = TensorDataset(features_test, labels_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)


In [13]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(features.shape[1], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
model = Net()

In [15]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)